In [73]:
import os

from QHyper.problems.algorithms.utils import WorkflowSchedulingSolverDecorator
from QHyper.problems.algorithms.workflow_decomposition import HeftBasedAlgorithm, add_entry_and_exit_tasks
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingOneHot
from QHyper.solvers import solver_from_config
from QHyper.solvers.gurobi import Gurobi
from reports.report import ExecutionReport, Solution

In [74]:
def get_solver(tasks_file: str, machines_file: str, deadline: int):
    solver_config = {
        "problem": {
            "type": "workflow_scheduling",
            "encoding": "one-hot",
            "tasks_file": tasks_file, 
            "machines_file": machines_file,
            "deadline": deadline,
        },
        "solver": {
            "type": "gurobi",
        }
    }

    return WorkflowSchedulingSolverDecorator(solver_from_config(solver_config))

def get_filename(tasks_file: str):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}.json"

def run_tests(test_data, save_dir):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        gurobi = get_solver(tasks_file, machines_file, deadline)
        schedule = gurobi.solve()
        solution: Solution = Solution("Gurobi")
        solution.add_part(schedule)
        report = ExecutionReport(
            workflow_file=tasks_file,
            machines_file=machines_file,
            deadline=deadline,
            solution=solution
        )
        report.write_json(os.path.join(save_dir, get_filename(tasks_file)))

In [75]:
tasks_files = [
    "workflows_data/workflows/srasearch_22_tasks.json", 
    "workflows_data/workflows/1000genome_156_tasks.json", 
    "workflows_data/workflows/1000genome_492_tasks.json", 
    "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json"
]

deadlines = [
    5000,
    5000,
    5000,
    5000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [76]:
run_tests(test_data, "reports/linear_machines/3000/hurbol")

Executing test for workflows_data/workflows/srasearch_22_tasks.json...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)
CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 110 columns and 410 nonzeros
Model fingerprint: 0x00cd4cd5
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+04]
  Objective range  [1e+00, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 239959.75087
Presolve removed 42 rows and 110 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 193627 239960 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.936265560320e+05, best bound 1.936

In [77]:
def divide_into_solvers(tasks_file: str, machines_file: str, deadline: int, n_parts: int):
    workflow = Workflow(tasks_file, machines_file, deadline)
    workflow = add_entry_and_exit_tasks(workflow)
    division = HeftBasedAlgorithm().decompose(workflow, n_parts)
    problems = map(lambda w: WorkflowSchedulingOneHot(w), division.workflows)
    return map(lambda p: WorkflowSchedulingSolverDecorator(Gurobi(p)), problems)

def get_filename(tasks_file: str):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}.json"

def run_tests_division(test_data, save_dir, n_parts):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        solvers = divide_into_solvers(tasks_file, machines_file, deadline, n_parts)
        solution = Solution(solver="Gurobi")
        schedules = map(lambda s: s.solve(), solvers)
        
        for s in schedules:
            solution.add_part(s)
        
        report = ExecutionReport(
            workflow_file=tasks_file,
            machines_file=machines_file,
            deadline=deadline,
            solution=solution
        )

        report.write_json(os.path.join(save_dir, get_filename(tasks_file)))

In [78]:
tasks_files = [
    "workflows_data/workflows/srasearch_22_tasks.json", 
    "workflows_data/workflows/1000genome_156_tasks.json", 
    "workflows_data/workflows/1000genome_492_tasks.json", 
    "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json"
]

deadlines = [
    5000,
    5000,
    5000,
    5000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [79]:
run_tests_division(test_data, "reports/linear_machines/3000/my", 2)

Executing test for workflows_data/workflows/srasearch_22_tasks.json...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)
CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 65 columns and 120 nonzeros
Model fingerprint: 0xdb4ecc1c
Variable types: 0 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [8e+01, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 249139.98695
Presolve removed 24 rows and 65 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 190251 249140 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.902507821760e+05, best bound 1.9025078